In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier


pd.set_option('display.max_columns', 500)

In [24]:
data = pd.read_csv("train.csv")
data.head()
#print(data.shape)

,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13
0,268055008619,Z,"Turner, Baldwin and Rhodes",4.250,214000,360,2012-03-01,05/2012,95,1.0,22.0,694.0,C86,30.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,672831657627,Y,"Swanson, Newton and Miller",4.875,144000,360,2012-01-01,03/2012,72,1.0,44.0,697.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,742515242108,Z,Thornton-Davis,3.250,366000,180,2012-01-01,03/2012,49,1.0,33.0,780.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,601385667462,X,OTHER,4.750,135000,360,2012-02-01,04/2012,46,2.0,44.0,633.0,B12,0.0,638.0,0.0,0,0,0,0,0,0,0,0,1,1,1,1,1
4,273870029961,X,OTHER,4.750,124000,360,2012-02-01,04/2012,80,1.0,43.0,681.0,C86,0.0,0.0,0.0,0,1,2,3,4,5,6,7,8,9,10,11,1


In [25]:
data.head()

int_cols = data.describe().columns
all_cols = data.columns
    
non_int_cols = [i for i in all_cols if i not in int_cols]
print(non_int_cols)
#data.source.unique()
for i in non_int_cols:
    tr = data[i]
    print(tr.unique())

['source', 'financial_institution', 'origination_date', 'first_payment_date', 'loan_purpose']
['Z' 'Y' 'X']
['Turner, Baldwin and Rhodes' 'Swanson, Newton and Miller'
 'Thornton-Davis' 'OTHER' 'Browning-Hart' 'Richardson Ltd'
 'Edwards-Hoffman' 'Richards-Walters' 'Martinez, Duffy and Bird'
 'Miller, Mcclure and Allen' 'Anderson-Taylor'
 'Taylor, Hunt and Rodriguez' 'Nicholson Group' 'Cole, Brooks and Vincent'
 'Sanchez, Hays and Wilkerson' 'Sanchez-Robinson' 'Suarez Inc'
 'Romero, Woods and Johnson' 'Chapman-Mcmahon']
['2012-03-01' '2012-01-01' '2012-02-01']
['05/2012' '03/2012' '04/2012' '02/2012']
['C86' 'B12' 'A23']


In [26]:
data['first_payment_date'] = pd.DatetimeIndex(data['first_payment_date']).month
data['origination_date'] = pd.DatetimeIndex(data['origination_date']).month

In [27]:
data.head()

,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13
0,268055008619,Z,"Turner, Baldwin and Rhodes",4.250,214000,360,3,5,95,1.0,22.0,694.0,C86,30.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,672831657627,Y,"Swanson, Newton and Miller",4.875,144000,360,1,3,72,1.0,44.0,697.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,742515242108,Z,Thornton-Davis,3.250,366000,180,1,3,49,1.0,33.0,780.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,601385667462,X,OTHER,4.750,135000,360,2,4,46,2.0,44.0,633.0,B12,0.0,638.0,0.0,0,0,0,0,0,0,0,0,1,1,1,1,1
4,273870029961,X,OTHER,4.750,124000,360,2,4,80,1.0,43.0,681.0,C86,0.0,0.0,0.0,0,1,2,3,4,5,6,7,8,9,10,11,1


In [28]:
data.isnull().any().any()

False

In [29]:
sample = pd.read_csv("sample_submission.csv")
sample.head()

,loan_id,m13
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [30]:
test = pd.read_csv("test.csv")
test.head()
print(test.shape)

(35866, 28)


In [31]:
int_cols = test.describe().columns
all_cols = test.columns
    
non_int_cols = [i for i in all_cols if i not in int_cols]
print(non_int_cols)
#data.source.unique()
for i in non_int_cols:
    tr = test[i]
    print(tr.unique())

['source', 'financial_institution', 'origination_date', 'first_payment_date', 'loan_purpose']
['Y' 'X' 'Z']
['Browning-Hart' 'OTHER' 'Miller, Mcclure and Allen'
 'Sanchez, Hays and Wilkerson' 'Thornton-Davis'
 'Swanson, Newton and Miller' 'Cole, Brooks and Vincent'
 'Sanchez-Robinson' 'Edwards-Hoffman' 'Turner, Baldwin and Rhodes'
 'Richardson Ltd' 'Taylor, Hunt and Rodriguez' 'Nicholson Group'
 'Suarez Inc' 'Martinez, Duffy and Bird' 'Anderson-Taylor'
 'Richards-Walters' 'Chapman-Mcmahon' 'Romero, Woods and Johnson']
['01/02/12' '01/01/12' '01/03/12']
['Apr-12' 'Mar-12' 'May-12' 'Feb-12']
['A23' 'C86' 'B12']


In [32]:
#test['first_payment_date'] = pd.DatetimeIndex(test['first_payment_date']).month

test['origination_date'] = pd.DatetimeIndex(test['origination_date']).month
first_payment_date_dict = {'Apr-12': 4, 'Mar-12':3, 'May-12':5, 'Feb-12':2} 
test['first_payment_date'] = test['first_payment_date'].map(first_payment_date_dict)

In [33]:
test.head()

,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12
0,1,Y,Browning-Hart,3.875,417000,360,1,4,75,1,20,790,A23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,X,OTHER,4.500,113000,360,1,4,80,2,33,793,C86,0,784,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,Y,OTHER,4.500,72000,360,1,3,75,1,34,710,C86,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,X,"Miller, Mcclure and Allen",4.125,123000,180,1,4,41,2,24,798,A23,0,813,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,X,Browning-Hart,3.250,166000,180,1,4,53,2,12,767,A23,0,768,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
scaler = StandardScaler()

def clean_train_data(data):
    int_cols = data.describe().columns
    all_cols = data.columns
    
    non_int_cols = [i for i in all_cols if i not in int_cols]
    print(non_int_cols)
    #data.source.unique()
    #for i in non_int_cols:
        #tr = data[i]
        #print(tr.unique())
    
    y = data['m13']
    X = data.drop(['m13'],axis=1)
    
    X['first_payment_date'] = pd.DatetimeIndex(X['first_payment_date']).month
    X['origination_date'] =  pd.DatetimeIndex(X['origination_date']).month
    
    X = pd.get_dummies(X,columns=['source', 'financial_institution', 'loan_purpose'])
    X_cols = X.columns
    df_X = scaler.fit_transform(X)
    df_scaled = pd.DataFrame(df_X,columns=X_cols)
    return df_scaled,y
    #return X,y


def clean_test_data(test):
    int_cols = data.describe().columns
    all_cols = data.columns
    
    non_int_cols = [i for i in all_cols if i not in int_cols]
    print(non_int_cols)
    #data.source.unique()
    #for i in non_int_cols:
        #tr = data[i]
        #print(tr.unique())
    
    #y = data['m13']
    #X = data.drop(['m13'],axis=1)
    #X = data.drop(['origination_date', 'first_payment_date'],axis=1)
    
    test['origination_date'] = pd.DatetimeIndex(test['origination_date']).month
    #first_payment_date_dict = {'Apr-12': 4, 'Mar-12':3, 'May-12':5, 'Feb-12':2} 
    #test['first_payment_date'] = test['first_payment_date'].map(first_payment_date_dict)
    test['first_payment_date'] = test['first_payment_date'] - test['origination_date']
    
    X = pd.get_dummies(test,columns=['source', 'financial_institution', 'loan_purpose'])
    X_cols = X.columns
    df_X = scaler.transform(X)
    df_scaled = pd.DataFrame(df_X,columns=X_cols)
    return df_scaled
    #return X
    

In [35]:
X,y = clean_train_data(data)
X.head()

['source', 'financial_institution', 'loan_purpose']


,loan_id,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,source_X,source_Y,source_Z,financial_institution_Anderson-Taylor,financial_institution_Browning-Hart,financial_institution_Chapman-Mcmahon,"financial_institution_Cole, Brooks and Vincent",financial_institution_Edwards-Hoffman,"financial_institution_Martinez, Duffy and Bird","financial_institution_Miller, Mcclure and Allen",financial_institution_Nicholson Group,financial_institution_OTHER,financial_institution_Richards-Walters,financial_institution_Richardson Ltd,"financial_institution_Romero, Woods and Johnson","financial_institution_Sanchez, Hays and Wilkerson",financial_institution_Sanchez-Robinson,financial_institution_Suarez Inc,"financial_institution_Swanson, Newton and Miller","financial_institution_Taylor, Hunt and Rodriguez",financial_institution_Thornton-Davis,"financial_institution_Turner, Baldwin and Rhodes",loan_purpose_A23,loan_purpose_B12,loan_purpose_C86
0,-1.083177,0.826517,0.050345,0.754428,0.0,0.0,1.594300,-1.207529,-0.898419,-1.798764,3.361200,-1.203344,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.039070,9.863609,-0.042652,-0.041791,-0.044583,-0.043642,-0.045276,-1.106044,-0.691643,2.631390,-0.064646,-0.615031,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,-0.865414,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,-0.075106,7.865748,-1.00749,-0.582239,1.764550
1,0.475125,2.182212,-0.560025,0.754428,0.0,0.0,0.264177,-1.207529,1.362454,-1.727692,-0.344138,-1.203344,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.039070,-0.041221,-0.042652,-0.041791,-0.044583,6.095719,-0.045276,-1.106044,1.445832,-0.380027,-0.064646,-0.615031,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,-0.865414,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,3.985426,-0.104723,-0.075106,-0.127133,-1.00749,1.717508,-0.566717
2,0.743392,-1.342595,1.375719,-1.250874,0.0,0.0,-1.065945,-1.207529,0.232018,0.238642,-0.344138,-1.203344,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.039070,-0.041221,-0.042652,-0.041791,-0.044583,-0.043642,-0.045276,-1.106044,-0.691643,2.631390,-0.064646,-0.615031,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,-0.865414,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,13.314522,-0.127133,-1.00749,1.717508,-0.566717
3,0.200074,1.911073,-0.638501,0.754428,0.0,0.0,-1.239439,0.828137,1.362454,-3.243901,-0.344138,0.467052,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.039070,-0.041221,-0.042652,7.756016,6.692613,6.095719,5.568680,0.904123,-0.691643,-0.380027,-0.064646,-0.615031,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,1.155516,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,-0.075106,-0.127133,-1.00749,1.717508,-0.566717
4,-1.060790,1.911073,-0.734416,0.754428,0.0,0.0,0.726829,-1.207529,1.259687,-2.106744,-0.344138,-1.203344,-0.057163,-0.050458,19.418760,36.047883,48.226620,48.36153,57.069319,59.387759,61.834283,62.340670,60.590177,61.349964,61.708245,0.904123,-0.691643,-0.380027,-0.064646,-0.615031,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,1.155516,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,-0.075106,-0.127133,-1.00749,-0.582239,1.764550


In [36]:
test_X = clean_test_data(test)
test_X.head()

['source', 'financial_institution', 'loan_purpose']


,loan_id,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,source_X,source_Y,source_Z,financial_institution_Anderson-Taylor,financial_institution_Browning-Hart,financial_institution_Chapman-Mcmahon,"financial_institution_Cole, Brooks and Vincent",financial_institution_Edwards-Hoffman,"financial_institution_Martinez, Duffy and Bird","financial_institution_Miller, Mcclure and Allen",financial_institution_Nicholson Group,financial_institution_OTHER,financial_institution_Richards-Walters,financial_institution_Richardson Ltd,"financial_institution_Romero, Woods and Johnson","financial_institution_Sanchez, Hays and Wilkerson",financial_institution_Sanchez-Robinson,financial_institution_Suarez Inc,"financial_institution_Swanson, Newton and Miller","financial_institution_Taylor, Hunt and Rodriguez",financial_institution_Thornton-Davis,"financial_institution_Turner, Baldwin and Rhodes",loan_purpose_A23,loan_purpose_B12,loan_purpose_C86
0,-2.11513,0.013100,1.820417,0.754428,0.0,2.0,0.437672,-1.207529,-1.103953,0.475550,-0.344138,-1.203344,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.03907,-0.041221,-0.042652,-0.041791,-0.044583,-0.043642,-0.045276,-1.106044,1.445832,-0.380027,-0.064646,1.625935,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,-0.865414,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,-0.075106,-0.127133,0.992566,-0.582239,-0.566717
1,-2.11513,1.368795,-0.830331,0.754428,0.0,2.0,0.726829,0.828137,0.232018,0.546622,-0.344138,0.849306,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.03907,-0.041221,-0.042652,-0.041791,-0.044583,-0.043642,-0.045276,0.904123,-0.691643,-0.380027,-0.064646,-0.615031,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,1.155516,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,-0.075106,-0.127133,-1.007490,-0.582239,1.764550
2,-2.11513,1.368795,-1.187834,0.754428,0.0,1.0,0.437672,-1.207529,0.334785,-1.419712,-0.344138,-1.203344,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.03907,-0.041221,-0.042652,-0.041791,-0.044583,-0.043642,-0.045276,-1.106044,1.445832,-0.380027,-0.064646,-0.615031,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,1.155516,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,-0.075106,-0.127133,-1.007490,-0.582239,1.764550
3,-2.11513,0.555378,-0.743136,-1.250874,0.0,2.0,-1.528596,0.828137,-0.692885,0.665076,-0.344138,0.925233,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.03907,-0.041221,-0.042652,-0.041791,-0.044583,-0.043642,-0.045276,0.904123,-0.691643,-0.380027,-0.064646,-0.615031,-0.057917,-0.119796,-0.208992,-0.205783,5.979170,-0.136275,-0.865414,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,-0.075106,-0.127133,0.992566,-0.582239,-0.566717
4,-2.11513,-1.342595,-0.368194,-1.250874,0.0,2.0,-0.834619,0.828137,-1.926088,-0.069338,-0.344138,0.807415,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.03907,-0.041221,-0.042652,-0.041791,-0.044583,-0.043642,-0.045276,0.904123,-0.691643,-0.380027,-0.064646,1.625935,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,-0.865414,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,-0.075106,-0.127133,0.992566,-0.582239,-0.566717


In [37]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=.33, random_state=99)


In [38]:
from sklearn.metrics import make_scorer

param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "base_estimator__max_depth" : [2, 4],
              "n_estimators": [2, 4, 6]
             }


DTC = DecisionTreeClassifier(random_state = 99, max_features = "auto")

ABC = AdaBoostClassifier(base_estimator = DTC)
scorer = make_scorer(f1_score)

# run grid search
cv_obj = GridSearchCV(ABC, param_grid=param_grid, cv=5, scoring=scorer)
cv = cv_obj.fit(X_train,y_train)
best_cv = cv.best_estimator_
print(best_cv)
train_pred = best_cv.predict(X_test)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=4,
                                                         max_features='auto',
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=99,
                             

clf1 = RandomForestClassifier(random_state=99)

model = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth=2), n_estimators = 4)

model.fit(X_train,y_train)
train_pred = model.predict(X_test)
score = (train_pred == y_test).mean()
print(score)
fscore = f1_score(train_pred,y_test,average='binary')
print(fscore)

clf = RandomForestClassifier(random_state=99)
#clf.fit(X_train,y_train)


param_grid = {
    'bootstrap': [True],
    #'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3]
    #'min_samples_leaf': [3, 4, 5],
    #'min_samples_split': [8, 10, 12],
    #'n_estimators': [100, 200, 300, 1000]
}

cv = GridSearchCV(clf, param_grid=param_grid)
cv.fit(X_train,y_train)
cv.best_params_

In [39]:
#train_pred = model.predict(X_test)
score = (train_pred == y_test).mean()
print(score)
fscore = f1_score(train_pred,y_test,average='binary')
print(fscore)


0.9944125326370757
0.3227848101265823


In [40]:
# Creating predict file
test_pred = best_cv.predict(test_X)
test_cols = test.columns
print(test_cols)

test_data1 = test.drop([ 'source', 'financial_institution', 'interest_rate',
       'unpaid_principal_bal', 'loan_term', 'origination_date',
       'first_payment_date', 'loan_to_value', 'number_of_borrowers',
       'debt_to_income_ratio', 'borrower_credit_score', 'loan_purpose',
       'insurance_percent', 'co-borrower_credit_score', 'insurance_type', 'm1',
       'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12'],axis=1)
test_pred_df = pd.DataFrame(test_pred, columns = ['m13'])
final_pred = pd.concat([test_data1,test_pred_df],axis=1)
final_pred.to_csv("predicts/predict_test112.csv", sep =',', encoding='utf-8',index=False)

Index(['loan_id', 'source', 'financial_institution', 'interest_rate',
       'unpaid_principal_bal', 'loan_term', 'origination_date',
       'first_payment_date', 'loan_to_value', 'number_of_borrowers',
       'debt_to_income_ratio', 'borrower_credit_score', 'loan_purpose',
       'insurance_percent', 'co-borrower_credit_score', 'insurance_type', 'm1',
       'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12'],
      dtype='object')


In [41]:
final_pred.m13.value_counts()

0    35779
1       87
Name: m13, dtype: int64